https://edinburghcyclehire.com/
Souřadnnice sanic jsou uváděny na 14 respektive 16 desetinných míst (Decimal degrees in WGS84).



In [ ]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
from bokeh.io import output_notebook, show
from bokeh.models import WMTSTileSource
from bokeh.plotting import figure, ColumnDataSource
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 500)
pd.set_option("display.precision", 14)
output_notebook()

In [ ]:
# vyloučit 3 "jízdy" s end stanicí 280 (Exhibition Centre Liverpool)
rides = pd.read_csv("201809-202010.csv", parse_dates=[0,1], usecols = ["started_at","ended_at","duration","start_station_id", "end_station_id"])    

In [568]:
# Vytvoření tabulky obsahující stanice = načtení z csv a dopočítání vzdálenosti mezi stanicemi.
# Zdrojová data obsahují nekonzistence v názvech a popisech stanic. Jejich zeměpisná šířka a délka také není u všech údajů stejná. To přičítám odchylce GPS a různému umístění kola.
# Vymazána ? koncová stanice id 280 - Exhibition Centre Liverpool (výstava kol).
# Celková tabulka stations je vytvořena spojením informací ze "start" a "end" slouců. Duplicity jsou řešeny vybráním prvního popisu a průměrem souřadnic - vše dle id.
# Další nekonzistencí je existence více stanic na jednom místě. Vyberu ty, které mají více použití.
# Načtení informací o "start" a "end" stanicích:
# Některé stanice jsou dočasné během akcí - zatím njsou dohromady s ostatními
stations = pd.read_csv("201809-202010.csv", index_col = "start_station_id", usecols = ["start_station_id","start_station_name","start_station_description","start_station_latitude", "start_station_longitude"])   
end_stations = pd.read_csv("201809-202010.csv", index_col = "end_station_id", usecols = ["end_station_id","end_station_name","end_station_description","end_station_latitude", "end_station_longitude"])  
# Přejmenování sloupců a indexu:
column_names = {"start_station_name":"name","start_station_description":"description","start_station_latitude":"latitude", "start_station_longitude":"longitude"}
stations.rename(columns=column_names, index={"start_station_id": "idd"}, inplace=True)
stations.index.names = ['id']
end_column_names = {"end_station_name":"name","end_station_description":"description","end_station_latitude":"latitude", "end_station_longitude":"longitude"}
end_stations.rename(columns=end_column_names, index={'end_station_id': 'id'}, inplace=True)
end_stations.index.names = ['id']
# Sloučení informací o "start" a "end" stanicích a jejich setřídění:
stations = stations.append(end_stations)
stations.sort_index(axis="index", inplace = True)
# Výpočet průměrných koorinátů a přidání k prvním názvům a popisům stanic:
stations_coord_mean = stations.groupby(["id"]).mean()
stations = stations.groupby(["id"]).first()
stations["latitude"] = stations_coord_mean["latitude"]
stations["longitude"] = stations_coord_mean["longitude"]
# Vymazána koncová stanice id 280 - Exhibition Centre Liverpool (výstava kol). Lepším řešením je vymazat "stanice", které jsou vzdáleny od Edinburgu víc než 50? km.
# stations.drop(280, inplace=True)
# přepočet wgs84 souřadnic na  Mercator projekci pro použití v WMTS tile map v openstreetmap.org 
stations["mer_x"] = stations["longitude"] * (6378137 * np.pi/180.0)
stations["mer_y"] = np.log(np.tan((90 + stations["latitude"]) * np.pi/360.0)) * 6378137
# Vymazání dočasných df:
end_stations = end_stations[0:0]
stations_coord_mean = stations_coord_mean[0:0]

In [578]:
# První použití stanic:
stations["first_ride"] = rides.groupby(["start_station_id"]).min()[["started_at"]] 
stations["first_ride_tmp"] = rides.groupby(["end_station_id"]).min()[["started_at"]] 
mfilter = (stations["first_ride"] > stations["first_ride_tmp"]) | (stations["first_ride"].isnull())
stations.loc[mfilter, "first_ride"] = stations.loc[mfilter, "first_ride_tmp"]
stations.drop("first_ride_tmp", axis=1, inplace=True)
#Počet použití stanic:
stations["number_of_rides"] = rides.groupby(["start_station_id"])[["duration"]].count()
stations["number_of_rides"].fillna(0, inplace = True)
stations["number_of_rides_tmp"] = rides.groupby(["end_station_id"])[["duration"]].count()
stations["number_of_rides_tmp"].fillna(0, inplace = True)
stations["number_of_rides"] = stations["number_of_rides"].astype(int) + stations["number_of_rides_tmp"].astype(int)
stations.drop("number_of_rides_tmp", axis=1, inplace=True)
#vzdálenost mezi stanicemi
# dx = 2*pi*R/360*(l2-l1); l = odmocnina(d1*di+ d2*d2)
for row2 in range(len(stations.index)): 
    stations[stations.index[row2]] = -1
    lat2 = stations.iloc[row2, 2]
    lon2 = stations.iloc[row2, 3]
    for row1 in range(len(stations.index)):
        if row1 > row2 :                       #počítám pouze první část vzdáleností - je to rychlejší a lépe se pak vyhledává 
            lat1 = stations.iloc[row1, 2]
            lon1 = stations.iloc[row1, 3]
            distance = round(np.sqrt(np.square((lat2 - lat1) * 2 * np.pi * 6378137 / 360) + np.square((lon2 - lon1) * 2 * np.pi * 6378137 / 360)))
            stations.iloc[row1, row2+8] = distance.astype(int)

In [581]:
# Stanice, které jsou blíže než x metrů od sebe:
s = []
for col in range(8,len(stations.index)):
    for row in range(len(stations.index)):
        if (stations.iloc[row, col] < 10) & (stations.iloc[row, col] > 0): 
            s.append(row)
stations.iloc[s, 0:8]      

,name,description,latitude,longitude,mer_x,mer_y,first_ride,number_of_rides
id,,,,,,,,
242,Virtual Depot,Virtual Depot,55.97240196356882,-3.15579403510969,-351301.38503686123295,7.55292362813887e+06,2018-11-08 09:05:20.220000+00:00,2
299,Depot Virtual,Depot internal station,55.97233473082817,-3.15578160260600,-351300.00105688051553,7.55291025356288e+06,2019-08-01 15:29:04.840000+00:00,8
299,Depot Virtual,Depot internal station,55.97233473082817,-3.15578160260600,-351300.00105688051553,7.55291025356288e+06,2019-08-01 15:29:04.840000+00:00,8
1769,Brunswick Place,Corner of Elm Row/Brunswick Street,55.96085174106757,-3.18098576246878,-354105.71529867721256,7.55062628898159e+06,2019-12-19 13:56:29.371000+00:00,7447
1753,Waitrose Comely Bank,On Fettes Avenue next to Waitrose,55.95953624971982,-3.22343440233871,-358831.07627386576496,7.55036468125536e+06,2019-11-21 17:37:34.828000+00:00,3991
1767,Bruntsfield Links,Corner of Bruntsfield Links next to Public Toilets,55.93712323998618,-3.20643187939306,-356938.36407735419925,7.54590884043149e+06,2019-12-12 16:35:17.438000+00:00,7537
1024,Meadow Place,End of Meadow Place,55.94023763167967,-3.19464021418406,-355625.72191068425309,7.54652784592206e+06,2019-07-30 10:07:54.082000+00:00,13709
1051,Warrender Park Road,On corner with Spottiswoode Street,55.93836921723931,-3.19803268854640,-356003.37042922846740,7.54615648049602e+06,2019-09-05 12:14:45.265000+00:00,4755
1744,Morningside Road,"Corner of Morningside Park, beside M&S",55.92798534979170,-3.20974977194460,-357307.71018669899786,7.54409291283158e+06,2019-11-07 14:47:07.713000+00:00,5277


In [ ]:
stations.describe

In [ ]:
stations.iloc[:, lambda stations: range(8,100)]

In [ ]:
# Stanice, které jsou blíže než x metrů od sebe:
s = []
for col in range(8,len(stations.index)):
    for row in range(len(stations.index)):
        if (stations.iloc[row, col] < 10) & (stations.iloc[row, col] > 0): 
            s.append(row)
stations.iloc[s]      

In [ ]:
s

In [ ]:
stations.iloc[s]  

In [ ]:
query = (stations.iloc[:, range(8, len(stations.index))]  < 10) & (stations.iloc[:, range(8, len(stations.index))]  > -1 )

In [ ]:
query

In [599]:
# openstreetmap používá mercator coordináty
TOOLTIPS = [("id", "@{id}"), ("(x,y)", "(@{latitude}, @{longitude})"), ("name", "@{name}"), ("desc", "@{description}"), ("rides", "@{number_of_rides}"),]
p = figure(plot_width=1200, plot_height=700, tools='reset, pan, wheel_zoom, save, box_zoom', x_range=(-380000,-340000), y_range=(7545000,7553000), 
           x_axis_type="mercator", y_axis_type="mercator", tooltips=TOOLTIPS, title="Edinburg - Just Eat Cycles") # Edinburg
p.add_tile(WMTSTileSource(url="https://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"))
p.circle("mer_x", "mer_y", size=8, color="red", alpha=1, source=ColumnDataSource(data=stations.iloc[:,0:8]))    # vybírám pouze část sloupců, protože bokeh nemůže mít názvy sloupců čísla !!!
show(p)

In [ ]:
stations.index

In [ ]:
query = (stations.iloc[:, range(8, len(stations.index))]  < 10) & (stations.iloc[:, range(8, len(stations.index))]  > -1 )

In [ ]:
data = pd.DataFrame(stations.iloc[s])

In [ ]:
stations.dtypes

In [592]:
stations.iloc[s, 0:8].dtypes

name                object
description         object
latitude           float64
longitude          float64
mer_x              float64
mer_y              float64
first_ride          object
number_of_rides      int32
dtype: object

In [601]:
# openstreetmap používá mercator coordináty ---- tady upravuji
data = stations.iloc[s, 0:8] 
TOOLTIPS = [("id", "@{id}"), ("(x,y)", "(@{latitude}, @{longitude})"), ("name", "@{name}"), ("desc", "@{description}"), ("rides", "@{number_of_rides}") ]
p = figure(plot_width=1200, plot_height=700, tools='reset, pan, wheel_zoom, save, box_zoom', x_range=(-380000,-340000), y_range=(7545000,7553000), 
           x_axis_type="mercator", y_axis_type="mercator", tooltips=TOOLTIPS, title="Edinburg - Just Eat Cycles") # Edinburg
p.add_tile(WMTSTileSource(url="https://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"))
#p.scatter("mer_x", "mer_y",  size=8, color="red", alpha=1, source=ColumnDataSource(data=stations))
p.circle("mer_x", "mer_y", size=8, color="red", alpha=1, source=ColumnDataSource(data=data ))
show(p)